In [216]:

from flask import jsonify
from dotenv import load_dotenv
import os
import pandas as pd
import numpy as np
from web3 import Web3
import requests

load_dotenv()

True

In [217]:
def query_txn_address(address, start_block=0):
    return (
        f"https://api-sepolia.etherscan.io/api?module=account&action=txlist&address={
            address}&"
        # f"https://api.etherscan.io/api?module=account&action=txlist&address={
        #     address}&"
        f"startblock={
            start_block}&endblock=19999999&page=1&offset=10000&sort=asc&"
        f"apikey={os.getenv('ETHERSCAN_API_KEY')}"
    )

In [218]:
def fetch_data(url):
    res = requests.get(url)
    res.raise_for_status()
    return res.json()['result']

In [219]:
def convert_columns(df, int_columns, float_columns):
    df[int_columns] = df[int_columns].astype(np.int64).fillna(0)
    df[float_columns] = df[float_columns].astype(np.float64).fillna(0)
    return df

In [220]:

def get_address_stats_normal_tnx(sample_df, address):
    address = address.lower()

    sample_df['eth_value'] = sample_df['value'].apply(lambda x: Web3.from_wei(int(x), 'ether'))

    sample_df['txn_type'] = np.where(
        sample_df['from'].str.lower() == address, 'sent', 'received')

    sample_df['unix time difference'] = sample_df['timeStamp'].diff()
    sample_df_time_dim = sample_df.groupby(
        'txn_type')['unix time difference'].sum()/60

    # Group by 'txn_type'
    sample_df_grouped = sample_df.groupby('txn_type')

    # Initialize statistics for sent transactions
    sent_stats = {
        'min_gas_fee': 0,
        'max_gas_fee': 0,
        'avg_gas_fee': 0,
        'total_ether_sent': 0,
        'unique_sent_to_addresses': 0,
        'sent_tnx': 0,
        'min_val_sent': 0,
        'max_val_sent': 0,
        'avg_val_sent': 0,
        'avg_min_between_sent_tnx': 0
    }

    if 'sent' in sample_df_grouped.groups:
        sent_df = sample_df_grouped.get_group('sent')
        sent_df.loc[:, 'gas_fee'] = sent_df['gasPrice'] * sent_df['gasUsed']
        sent_df['gas_fee_eth'] = sent_df['gas_fee'].apply(
            lambda x: Web3.from_wei(int(x), 'ether'))
        sent_stats['avg_gas_fee'] = sent_df['gas_fee_eth'].mean()
        sent_stats['total_ether_sent'] = sent_df['eth_value'].sum()
        sent_stats['unique_sent_to_addresses'] = sent_df['to'].nunique()
        sent_stats['sent_tnx'] = len(sent_df)
        sent_stats['min_val_sent'] = sent_df['eth_value'].min()
        sent_stats['max_val_sent'] = sent_df['eth_value'].max()
        sent_stats['avg_val_sent'] = sent_df['eth_value'].mean()

        sent_stats['avg_min_between_sent_tnx'] = sample_df_time_dim['sent'] / \
            sent_stats['sent_tnx']

    # Initialize statistics for received transactions
    received_stats = {
        'received_tnx': 0,
        'min_value_received': 0,
        'max_value_received': 0,
        'avg_value_received': 0,
        'total_ether_received': 0,
        'unique_received_from_addresses': 0,
        'avg_min_between_received_tnx': 0
    }

    if 'received' in sample_df_grouped.groups:
        received_df = sample_df_grouped.get_group('received')

        received_stats['received_tnx'] = len(received_df)
        received_stats['min_value_received'] = received_df['eth_value'].min()
        received_stats['max_value_received'] = received_df['eth_value'].max()
        received_stats['avg_value_received'] = received_df['eth_value'].mean()
        received_stats['total_ether_received'] = received_df['eth_value'].sum()
        received_stats['unique_received_from_addresses'] = received_df['from'].nunique()
        received_stats['avg_min_between_received_tnx'] = received_df['timeStamp'].diff(
        ).sum() / 60 / len(received_df)

    # Compile overall statistics
    overall_stats = {
        'address': address,
        'avg_sent_time': sent_stats['avg_min_between_sent_tnx'],
        'avg_received_time': received_stats['avg_min_between_received_tnx'],
        'time_difference_mins': (sample_df['timeStamp'].max() - sample_df['timeStamp'].min()) / 60,
        'sent': sent_stats['sent_tnx'],
        'received': received_stats['received_tnx'],
        'errors': len(sample_df[sample_df['isError'] != 0]),
        'unique_received_addresses': received_stats['unique_received_from_addresses'],
        'unique_sent_addresses': sent_stats['unique_sent_to_addresses'],
        'min_eth_received': received_stats['min_value_received'],
        'max_eth_received': received_stats['max_value_received'],
        'avg_eth_received': received_stats['avg_value_received'],
        'min_eth_sent': sent_stats['min_val_sent'],
        'max_eth_sent': sent_stats['max_val_sent'],
        'avg_eth_sent': sent_stats['avg_val_sent'],
        'avg_gas_fee': sent_stats['avg_gas_fee'],
        'total_txs': len(sample_df),
        'total_eth_sent': sent_stats['total_ether_sent'],
        'total_eth_received': received_stats['total_ether_received'],
    }

    return pd.DataFrame([overall_stats])

In [221]:
def get_tx_by_address(address):
    try:
        columns_to_keep = ['from', 'to', 'contractAddress']
        int_columns = ['blockNumber', 'timeStamp', 'isError']
        float_columns = ['value', 'gasPrice',
                         'gas', 'cumulativeGasUsed', 'gasUsed']

        initial_url = query_txn_address(address)
        data = fetch_data(initial_url)

        df = pd.DataFrame(data)[columns_to_keep + int_columns + float_columns]

        # Loop through the data
        if len(df) == 10000:
            last_block_number = df['blockNumber'].iloc[-1]
            additional_url = query_txn_address(address, last_block_number)
            additional_data = fetch_data(additional_url)

            new_df = pd.DataFrame(additional_data)[
                columns_to_keep + int_columns + float_columns]
            df = pd.concat([df, new_df], ignore_index=True).drop_duplicates()

        df = convert_columns(df, int_columns, float_columns)

        return df

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return pd.DataFrame()

In [222]:
address = '0x087791512beF6469B7ea2799a55D508a9bf6be33'

In [223]:

sample_df = get_tx_by_address(address)

In [224]:
address = address.lower()

sample_df['eth_value'] = sample_df['value'].apply(
        lambda x: Web3.from_wei(int(x), 'ether'))

sample_df['txn_type'] = np.where(
        sample_df['from'].str.lower() == address, 'sent', 'received')

sample_df['unix time difference'] = sample_df['timeStamp'].diff()
sample_df_time_dim = sample_df.groupby('txn_type')['unix time difference'].sum()/60

    # Group by 'txn_type'
sample_df_grouped = sample_df.groupby('txn_type')

    # Initialize statistics for sent transactions
sent_stats = {
    'avg_min_between_sent_tnx': 0
    }

if 'sent' in sample_df_grouped.groups:
    sent_df = sample_df_grouped.get_group('sent')
    sent_stats['avg_min_between_sent_tnx'] = sample_df_time_dim['sent']/len(sent_df)
    # sent_stats['avg_min_between_sent_tnx'] = sent_df['timeStamp'].diff().sum()

received_stats = {
    'avg_min_between_received_tnx': 0
    }

if 'received' in sample_df_grouped.groups:
    received_df = sample_df_grouped.get_group('received')

    received_stats['avg_min_between_received_tnx'] = sample_df_time_dim['received'] / len(received_df)
    # received_stats['avg_min_between_received_tnx'] = received_df['timeStamp'].diff().sum()

    # Compile overall statistics
overall_stats = {
    'avg_sent_time': sent_stats['avg_min_between_sent_tnx'],
    'avg_received_time': received_stats['avg_min_between_received_tnx'],
    'time_difference_mins': (sample_df['timeStamp'].max() - sample_df['timeStamp'].min()) / 60,
    }

In [225]:
sent_df

,from,to,contractAddress,blockNumber,timeStamp,isError,value,gasPrice,gas,cumulativeGasUsed,gasUsed,eth_value,txn_type,unix time difference
2,0x087791512bef6469b7ea2799a55d508a9bf6be33,0x5e1341d31930496ea2c58c59e79b417e4ea57343,,5073342,1705077684,0,6.000000e+17,1.669953e+11,21000.0,2110552.0,21000.0,0.6,sent,89064.0
6,0x087791512bef6469b7ea2799a55d508a9bf6be33,0x8626f6940e2eb28930efb4cef49b2d1f2c9c1199,,5084450,1705233780,0,6.000000e+17,9.607841e+09,21000.0,14139024.0,21000.0,0.6,sent,155556.0
8,0x087791512bef6469b7ea2799a55d508a9bf6be33,0x8626f6940e2eb28930efb4cef49b2d1f2c9c1199,,5084589,1705235772,0,4.000000e+17,1.976354e+10,21000.0,8248337.0,21000.0,0.4,sent,912.0


In [226]:
received_df

,from,to,contractAddress,blockNumber,timeStamp,isError,value,gasPrice,gas,cumulativeGasUsed,gasUsed,eth_value,txn_type,unix time difference
0,0xa7e4ef0a9e15bdef215e2ed87ae050f974ecd60b,0x087791512bef6469b7ea2799a55d508a9bf6be33,,4084822,1691980116,0,5.000000e+17,7.231586e+08,63000.0,13261638.0,21000.0,0.5,received,NaN
1,0x1fc35b79fb11ea7d4532da128dfa9db573c51b09,0x087791512bef6469b7ea2799a55d508a9bf6be33,,5066990,1704988620,0,5.000000e+17,7.018743e+10,22000.0,5154056.0,21000.0,0.5,received,13008504.0
3,0x1fc35b79fb11ea7d4532da128dfa9db573c51b09,0x087791512bef6469b7ea2799a55d508a9bf6be33,,5073351,1705077804,0,5.000000e+17,1.375728e+11,22000.0,3488773.0,21000.0,0.5,received,120.0
4,0x6cc9397c3b38739dacbfaa68ead5f5d77ba5f455,0x087791512bef6469b7ea2799a55d508a9bf6be33,,5073378,1705078164,0,6.785320e+16,1.039942e+11,21000.0,2192745.0,21000.0,0.067853205,received,360.0
5,0x4281ecf07378ee595c564a59048801330f3084ee,0x087791512bef6469b7ea2799a55d508a9bf6be33,,5073380,1705078224,0,2.500000e+17,1.003857e+11,60000.0,29200758.0,21000.0,0.25,received,60.0
7,0x7ed746476a7f6520babd24eee1fdbcd0f7fb271f,0x087791512bef6469b7ea2799a55d508a9bf6be33,,5084530,1705234860,0,5.000000e+17,3.342716e+10,63000.0,294000.0,21000.0,0.5,received,1080.0
9,0x3c352ea32dfbb757ccdf4b457e52daf6ecc21917,0x087791512bef6469b7ea2799a55d508a9bf6be33,,5113765,1705659948,0,5.000000e+17,2.197564e+11,63000.0,42000.0,21000.0,0.5,received,424176.0
10,0x1fc35b79fb11ea7d4532da128dfa9db573c51b09,0x087791512bef6469b7ea2799a55d508a9bf6be33,,5145691,1706102424,0,5.000000e+17,6.605595e+10,22000.0,1995955.0,21000.0,0.5,received,442476.0
11,0xd1bd27c9be2943e8ec0ce43d6f8b8f9ce434eeb7,0x087791512bef6469b7ea2799a55d508a9bf6be33,,5782075,1714140720,0,1.000000e+16,1.427894e+09,1000000.0,8917881.0,21000.0,0.01,received,8038296.0
